In [1]:
import sys
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
#get_ipython().run_line_magic('matplotlib', 'inline')
import re
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.pipeline import Pipeline
import seaborn as sns
import pickle

In [2]:
f = open('nlp_train.json') 
import pandas as pd
import json
data = json.load(f) 

    # Iterating through the json 
    # list 
    #print(data["fkrr36o"]["emotion"])
f.close() 
fl=[]
    #print(data["fkrr36o"])
for u,v in data.items():
    l=[]
    l.append(u)
    l.append(v["body"])
    l.append(1 if v["emotion"]["anger"]==True else 0)
    l.append(1 if v["emotion"]["anticipation"]==True else 0)
    l.append(1 if v["emotion"]["disgust"]==True else 0)
    l.append(1 if v["emotion"]["fear"]==True else 0)
    l.append(1 if v["emotion"]["joy"]==True else 0)
    l.append(1 if v["emotion"]["love"]==True else 0)
    l.append(1 if v["emotion"]["optimism"]==True else 0)
    l.append(1 if v["emotion"]["pessimism"]==True else 0)
    l.append(1 if v["emotion"]["sadness"]==True else 0)
    l.append(1 if v["emotion"]["surprise"]==True else 0)
    l.append(1 if v["emotion"]["trust"]==True else 0)
    l.append(1 if v["emotion"]["neutral"]==True else 0)
    fl.append(l)

df = pd.DataFrame(fl, columns=["id","body","anger","anticipation","disgust","fear","joy","love","optimism","pessimism","sadness","surprise","trust","neutral"])
df.to_pickle("./dataframe.pkl")
print("Data loaded to the dataframe")

Data loaded to the dataframe


In [3]:
df = pd.read_pickle("./dataframe.pkl")
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
porter = PorterStemmer()
def stemSentence(sentence):
    token_words=word_tokenize(sentence)
    token_words
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)


In [4]:
def clean(data):
    data = data.lower()
    data = re.sub('<[^<]+?>', '', data)
    data=re.sub('[!#?,.:";]', '', data)
    data=re.sub(r'[0-9]+', '', data)
    data= re.sub("i'm","i am",data)
    data = stemSentence(data)
    return data

In [5]:
print("Preprocessing started")
df['body'] = df['body'].map(lambda x : clean(x))
print("Preprocessing completed")


Preprocessing started
Preprocessing completed


In [6]:


emotions = ["anger","anticipation","disgust","fear","joy","love","optimism","pessimism","sadness","surprise","trust","neutral"]
train, test = train_test_split(df, random_state=40, test_size=0.25, shuffle=True)

X_train= train['body']
X_test= test['body']
print(X_train.shape)
print(X_test.shape)

(1119,)
(374,)


In [17]:
NB= Pipeline([('tfidf', TfidfVectorizer(stop_words=stop_words)),('clf', OneVsRestClassifier(MultinomialNB(fit_prior=True, class_prior=None))),])
for emotion in emotions:
    NB.fit(X_train, train[emotion])
    prediction = NB.predict(X_test)
    print('Test accuracy of :'+emotion)
    print(accuracy_score(test[emotion], prediction))


Test accuracy of :anger
0.6737967914438503
Test accuracy of :anticipation
0.5588235294117647
Test accuracy of :disgust
0.7406417112299465
Test accuracy of :fear
0.6818181818181818
Test accuracy of :joy
0.8609625668449198
Test accuracy of :love
0.9090909090909091
Test accuracy of :optimism
0.6149732620320856
Test accuracy of :pessimism
0.6684491978609626
Test accuracy of :sadness
0.7165775401069518
Test accuracy of :surprise
0.8877005347593583
Test accuracy of :trust
0.8342245989304813
Test accuracy of :neutral
0.8689839572192514


In [19]:
SVC = Pipeline([('tfidf', TfidfVectorizer(stop_words=stop_words)),('clf', OneVsRestClassifier(LinearSVC(penalty='l2', loss='squared_hinge', dual=True, tol=0.0001, C=1.0, multi_class='ovr', fit_intercept=True, intercept_scaling=1, class_weight=None, verbose=0, random_state=None, max_iter=1000), n_jobs=1)),])
for emotion in emotions: 
    SVC.fit(X_train, train[emotion])
    prediction = SVC.predict(X_test)
    print('Test accuracy of :'+emotion)
    print(accuracy_score(test[emotion], prediction))


Test accuracy of :anger
0.7807486631016043
Test accuracy of :anticipation
0.6470588235294118
Test accuracy of :disgust
0.7459893048128342
Test accuracy of :fear
0.7165775401069518
Test accuracy of :joy
0.8582887700534759
Test accuracy of :love
0.9037433155080213
Test accuracy of :optimism
0.7058823529411765
Test accuracy of :pessimism
0.660427807486631
Test accuracy of :sadness
0.7433155080213903
Test accuracy of :surprise
0.8823529411764706
Test accuracy of :trust
0.839572192513369
Test accuracy of :neutral
0.8663101604278075


In [20]:
pac = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(PassiveAggressiveClassifier(C=1.0, fit_intercept=True, max_iter=1000, tol=0.001, early_stopping=False, validation_fraction=0.1, n_iter_no_change=5, shuffle=True, verbose=0, loss='hinge', n_jobs=None, random_state=None, warm_start=False, class_weight=None, average=False))),
            ])

for emotion in emotions:
    pac.fit(X_train, train[emotion])
    prediction = pac.predict(X_test)
    print('Test accuracy of :'+emotion)
    print(accuracy_score(test[emotion], prediction))

Test accuracy of :anger
0.7807486631016043
Test accuracy of :anticipation
0.6149732620320856
Test accuracy of :disgust
0.7219251336898396
Test accuracy of :fear
0.6951871657754011
Test accuracy of :joy
0.8502673796791443
Test accuracy of :love
0.8957219251336899
Test accuracy of :optimism
0.6844919786096256
Test accuracy of :pessimism
0.6443850267379679
Test accuracy of :sadness
0.7245989304812834
Test accuracy of :surprise
0.8823529411764706
Test accuracy of :trust
0.8048128342245989
Test accuracy of :neutral
0.8582887700534759


In [23]:
import pickle
pkl_filename = "pickle_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(SVC, file)

In [12]:
print("Model Saved")

Model Saved
